In [ ]:
#default_exp medical.imaging

In [ ]:
#export 
from local.torch_basics import *
from local.test import *
from local.core import *
from local.layers import *
from local.data.all import *
from local.optimizer import *
from local.learner import *
from local.metrics import *
from local.vision import models

import pydicom
from pydicom.dataset import Dataset as DcmDataset
from pydicom.tag import BaseTag as DcmTag
from pydicom.multival import MultiValue as DcmMultiValue

In [ ]:
from local.notebook.showdoc import *

In [ ]:
#export
_all_ = ['DcmDataset', 'DcmTag', 'DcmMultiValue']

In [ ]:
#export
def _split_elem(res,k,v):
    if not isinstance(v,DcmMultiValue): return
    res[f'Multi{k}'] = 1
    for i,o in enumerate(v): res[f'{k}{"" if i==0 else i}']=o

In [ ]:
#export
@patch
def dcmread(self:Path): return pydicom.dcmread(str(self))

In [ ]:
#export
@patch
def as_dict(self:DcmDataset, fname, px_summ=True):
    pxdata = (0x7fe0,0x0010)
    vals = [self[o] for o in self.keys() if o != pxdata]
    its = [(v.keyword,v.value) for v in vals]
    res = dict(its)
    res['fname'] = fname
    for k,v in its: _split_elem(res,k,v)
    if not px_summ: return res
    stats = 'min','max','mean','std'
    try:
        pxs = self.pixel_array
        for f in stats: res['img_'+f] = getattr(pxs,f)()
    except Exception as e:
        for f in stats: res['img_'+f] = 0
        print(res,e)
    return res

In [ ]:
#export
def _dcm2dict(px_summ, fn):
    dcm = fn.dcmread()
    try: return dcm.as_dict(fn.name, px_summ=px_summ)
    finally: del(dcm)

In [ ]:
#export
@delegates(parallel)
def _from_dicoms(cls, fns, px_summ=True, **kwargs): return pd.DataFrame(parallel(partial(_dcm2dict,px_summ), fns, **kwargs))
pd.DataFrame.from_dicoms = classmethod(_from_dicoms)

In [ ]:
#export
@patch_property
def scaled_px(self:DcmDataset):
    "`pixel_array` scaled by `RescaleSlope` and `RescaleIntercept"
    img = self.pixel_array
    return img*self.RescaleSlope + self.RescaleIntercept

## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 01a_utils.ipynb.
Converted 01b_dispatch.ipynb.
Converted 01c_torch_core.ipynb.
Converted 02_script.ipynb.
Converted 03_dataloader.ipynb.
Converted 04_transform.ipynb.
Converted 05_data_core.ipynb.
Converted 06_data_transforms.ipynb.
Converted 07_vision_core.ipynb.
Converted 08_pets_tutorial.ipynb.
Converted 09_vision_augment.ipynb.
Converted 10_data_block.ipynb.
Converted 11_layers.ipynb.
Converted 11a_vision_models_xresnet.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 14_callback_schedule.ipynb.
Converted 14a_callback_data.ipynb.
Converted 15_callback_hook.ipynb.
Converted 16_callback_progress.ipynb.
Converted 17_callback_tracker.ipynb.
Converted 18_callback_fp16.ipynb.
Converted 19_callback_mixup.ipynb.
Converted 20_metrics.ipynb.
Converted 21_vision_learner.ipynb.
Converted 22_tutorial_imagenette.ipynb.
Converted 23_tutorial_transfer_learning.ipynb.
Converted 30_text_core.ipynb.
Converted 31_tex